In [491]:
from json import loads
from pandas import DataFrame, merge, read_excel
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException
from re import findall
from time import sleep

In [398]:
class ScraperFalabella:
    def __init__(self):
        chrome_options = ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        self._driver = Chrome(
            ChromeDriverManager().install(),
            options=chrome_options
        )
        self._wait = WebDriverWait(self._driver, 8)

    def enter_website(self, url):
        self._driver.get(url)
        self._driver.maximize_window()

    def get_element(self, selector, path):
        return self._wait.until(lambda x: x.find_element(selector, path))
    
    def get_elements(self, selector, path):
        return self._wait.until(lambda x: x.find_elements(selector, path))
        
    def close_popups(self):
        self.get_element(By.ID, "testId-accept-cookies-btn").click()
        #self.get_element(By.CLASS_NAME, "dy-lb-close").click()
        
    def get_category_info(self):
        self.get_element(By.CLASS_NAME, "TopMenu-module_categoryWrapper__Q_tEE").click()
        category_list = self.get_elements(By.XPATH, "//a[@class='SideMenu-module_itemWrapper__3IXOl']")
        category_links = []
        category_info = {
            "Name": [],
            "Link": []
        }
        for category in category_list:
            try:
                category.click()
                subcategory_list = self.get_elements(By.XPATH, "//a[@class='SubCategories-module_hover-effect__1E3TD SubCategories-module_list-title__3bskI']")
                for subcategory in subcategory_list:
                    if subcategory.get_attribute("href").find("category") == -1:
                        continue
                    subcategory_links.append(subcategory.get_attribute("href"))
            except TimeoutException as error:
                continue
        self._driver.get(subcategory_links[0])
        modal = self.get_element(By.ID, "testId-modal-close")
        modal.click()
        for link in subcategory_links:
            self._driver.get(link)
            no_error = True
            while no_error:
                try:
                    category = self.get_element(By.XPATH, "//a[@class='jsx-2883309125 l1category']")
                    name = category.text
                    category.click()
                except ElementNotInteractableException:
                    no_error = False
            category_info["Name"].append(name)
            category_info["Link"].append(self._driver.execute_script("return document.URL"))
        
        return DataFrame(category_info)
    
    def get_subcategory_info(self, category_links):
        subcategory_info = {
            "Link": [],
            "Subcategory_1": [],
            "Link_1": [],
        }
        for category_level in category_links:
            try:
                scraper._driver.get(category_level)
                data = self.get_element(By.XPATH, "//script[@id='__NEXT_DATA__']")
                data_json = loads(data.get_attribute("text"))
                filters_value = data_json["props"]["pageProps"]["facets"][:3]
                for filter_value in filters_value:
                    if filter_value["name"] == "Categoría":
                        data_values = filter_value["values"]
                        for item in data_values:
                            title = item["title"]
                            subcategory_info["Link_1"].append("https://tienda.falabella.com.pe/falabella-pe/category/" + item["id"] + "/" + title.replace(" ", "-"))
                            subcategory_info["Link"].append(category_level)
                            subcategory_info["Subcategory_1"].append(title)
                        break

            except (KeyError, IndexError) as error:
                print(error.__class__.__name__, category_level)
                
            except TimeoutException as error:
                print(error.__class__.__name__, category_level)
            
        return DataFrame(subcategory_info)

    def extract_categories(self):
        df_category = self.get_category_info()
        df_subcategory_1 = scraper.get_subcategory_info(df_category["Link"])
        df_subcategory_2 = scraper.get_subcategory_info(df_subcategory_1["Link_1"])
        df_subcategory_2.rename({"Link": "Link_1", "Subcategory_1":"Subcategory_2", "Link_1": "Link_2"}, axis=1,inplace=True)
        df_subcategory_3 = scraper.get_subcategory_info(df_subcategory_2["Link_2"])
        df_subcategory_3.rename({"Link": "Link_2", "Subcategory_1":"Subcategory_3", "Link_1": "Link_3"}, axis=1,inplace=True)
        df_subcategory_4 = scraper.get_subcategory_info(df_subcategory_3["Link_3"])
        df_subcategory_4.rename({"Link": "Link_3", "Subcategory_1":"Subcategory_4", "Link_1": "Link_4"}, axis=1,inplace=True)
        

In [448]:
scraper = ScraperFalabella()
scraper.enter_website("https://tienda.falabella.com.pe/falabella-pe")
scraper.close_popups()
#df_cat = scraper.get_category_info()
# df = scraper.get_subcategory_info(links)
# df_2 = scraper.get_subcategory_info(df["Link_1"])
# df_3 = scraper.get_subcategory_info(df_2["Link_2"])
# df_4 = scraper.get_subcategory_info(df_3["Link_3"])
# df_5 = scraper.get_subcategory_info(df_4["Link_4"])

KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14332/Artículos-para-el-cuidado-de-la-nariz
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14375/Glucómetros
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14336/Aceites-corporales
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14338/Anticonceptivos-para-Hombre
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14347/Lubricantes-y-humectantes
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14349/Potencializadores
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG33641/Equipamiento-medico
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG33642/Insumos-medicos
KeyError https://tienda.falabella.com.pe/falabella-pe/category/CATG14352/Equipos-de-movilidad


In [433]:
df_4.to_excel("subcategory_4.xlsx",encoding="utf-8-sig")

In [436]:
df_merge = merge(df_cat, df, left_on='Root', right_on='Link', how="left")
df_merge = merge(df_merge, df_2, left_on='Link_1', right_on='Link_1', how="left")
df_merge = merge(df_merge, df_3, left_on='Link_2', right_on='Link_2', how="left")
df_merge = merge(df_merge, df_4, left_on='Link_3', right_on='Link_3', how="left")
df_merge.drop("Root", axis = 1, inplace=True)

In [443]:
index_list = list(range(395, 509))
df_merge.drop(df_merge.index[index_list], inplace =True)

In [444]:
df_merge.reset_index(drop=True, inplace=True)

In [489]:
df_merge.to_excel("Falabella_category.xlsx",encoding="utf-8-sig")

In [508]:
df_fixed = read_excel("Falabella_category.xlsx")

In [509]:
df_fixed.drop("Unnamed: 0", axis=1,inplace=True)

In [265]:
df_sub_sub = df_sub_sub.dropna(how="any")

In [510]:
df_old = df_fixed.stack().groupby(level=0).apply(list).tolist()

In [511]:
whole_links = [x[-1] for x in df_old]

In [504]:
old_size = len(whole_links)
whole_links = list(set(whole_links))
new_size = len(whole_links)
print(old_size - new_size)

34


In [512]:
visited = set()
dup = {x for x in whole_links if x in visited or (visited.add(x) or False)}
dup

{'https://tienda.falabella.com.pe/falabella-pe/category/CATG14322/Termómetros',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14331/Artículos-para-dejar-de-fumar',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14332/Artículos-para-el-cuidado-de-la-nariz',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14333/Artículos-para-el-cuidado-de-las-orejas',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14334/Artículos-para-el-cuidado-de-los-ojos-y-lentes',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14336/Aceites-corporales',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14338/Anticonceptivos-para-Hombre',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14339/Anticonceptivos-para-Mujer',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14342/Estimuladores-Sexuales',
 'https://tienda.falabella.com.pe/falabella-pe/category/CATG14344/Juguetes-Sexuales',
 'https://tienda.falabella.com.pe/fala

In [519]:
for dups in list(dup):
    for item in df_old:
        if item[-1] == dups:
            print(item[0], item[2])
    print("----------------------------------------------------------------------")

Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
----------------------------------------------------------------------
Especiales Especial Tecnología
Belleza, higiene y salud Salud
-------